In [1]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np

In [ ]:
def char_to_weight(character):
    if character == "x":
        return np.array([0, 1, 0, 0])
    if character == "y":
        return np.array([0, 0, 1, 0])
    if character == "z":
        return np.array([0, 0, 0, 1])
    if character == "i":
        return np.array([1, 0, 0, 0])
def mapping_single_pauliword_one_gate(character, instructors):
    # Map a single Pauliword to list
    # Example: xyy -> [0, 1, 0, 0][0,0,1,0][0,0,1,0] -- h --> [0, 0, 0, 1][0,0,-1,0][0,0,-1,0] = z(-y)(-y)
    from numpy import sin, cos, sqrt
    weights = char_to_weight(character)

    for gate, index, param in instructors:
        I, A, B, C = weights
        if gate == "h":
            weights = np.array([I, 0, 0, 1])
        if gate == "s":
            weights = np.array([I, 0, 1, 0])
        if gate == "t":
            weights = np.array([I, 1 / sqrt(2), 1 / sqrt(2), 0])
        if gate == "rx":
            weights = np.array([I, A, B * cos(param) - C * sin(param), B * sin(param) + C * cos(param)])
        if gate == "ry":
            weights = np.array([I, A * cos(param) + C * sin(param), B, C * cos(param) - A * sin(param)])
        if gate == "rz":
            weights = np.array([I, A * cos(param) - B * sin(param), B * cos(param) + A * sin(param), C])
    return weights


In [ ]:
class Instructor:
    def __init__(self, num_qubits):
        self.clusters = []
        self.cluster = []
        self.cluster_temp = []
        self.xcluster = []
        self.xclusters = []
        self.instructors = []
        self.num_qubits = num_qubits
        self.barriers = [0] * num_qubits

    def append(self, gate, index, param=0):
        self.instructors.append([gate, index, param])

    def clustering(self):
        while len(self.instructors) > 0:
            gate, index, _ = self.instructors[0]
            sum_barrier = sum(self.barriers)
            if sum_barrier == self.num_qubits:
                self.clusters.append(self.cluster)
                self.cluster = []
                self.instructors = self.cluster_temp + self.instructors
                self.cluster_temp = []
                gate, index, _ = self.instructors[0]
                self.barriers = [0] * self.num_qubits

            if gate == "cx":
                self.barriers[index[0]] = 1
                self.barriers[index[1]] = 1
                self.xcluster.append(self.instructors[0])
                if sum(self.barriers) == self.num_qubits:
                    self.xclusters.append(self.xcluster[:-1])
                    self.xcluster = [self.xcluster[-1]]
                self.instructors.pop(0)
            else:
                if self.barriers[index] == 0:
                    self.cluster.append(self.instructors[0])
                    self.instructors.pop(0)
                else:
                    self.cluster_temp.append(self.instructors.pop(0))
        if len(self.cluster) > 0:
            self.clusters.append(self.cluster)
        if len(self.cluster_temp) > 0:
            self.clusters.append(self.cluster_temp)
        if len(self.xcluster) > 0:
            self.xclusters.append(self.xcluster)


ins = Instructor(4)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.clustering()

In [82]:
ins.clusters

[[['h', 0, 0],
  ['h', 1, 0],
  ['h', 2, 0],
  ['h', 0, 0],
  ['h', 2, 0],
  ['h', 2, 0],
  ['h', 3, 0],
  ['h', 3, 0],
  ['h', 3, 0],
  ['h', 3, 0],
  ['h', 3, 0]],
 [['h', 0, 0],
  ['h', 1, 0],
  ['h', 0, 0],
  ['h', 1, 0],
  ['h', 2, 0],
  ['h', 0, 0],
  ['h', 2, 0]]]

In [83]:
ins.xclusters

[[['cx', [0, 1], 0], ['cx', [1, 2], 0]], [['cx', [1, 3], 0]]]